In [30]:
# %%
import requests
import zipfile
import pandas as pd
import os
import re
import time
import gzip
from datetime import datetime, timedelta

# 轉換度、分為十進位格式
def to_decimal(d, m):
    return d + m / 60

ROOT = "./"

# ※ 原本下載 AMeDAS 站點清單與合併站點資料的部分保持不變
def read_csv_with_multiple_encodings(file_path, encodings=['cp932', 'utf-8', 'shift_jis', 'euc-jp']):
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"Successfully read the file with encoding: {encoding}")
            return df
        except UnicodeDecodeError as e:
            print(f"Failed to read with encoding {encoding}: {e}")
    raise ValueError("Failed to read the file with all provided encodings")

def download_amedas_station_list():
    AMeDAS_STA_list = "https://www.jma.go.jp/jma/kishou/know/amedas/ame_master.zip"
    r = requests.get(AMeDAS_STA_list)
    with open("ame_master.zip", "wb") as code:
        code.write(r.content)
    with zipfile.ZipFile("ame_master.zip", "r") as z:
        z.extractall("ame_master")

    # 找出解壓後唯一的 CSV 檔案
    for file in os.listdir("ame_master"):
        if file.endswith(".csv"):
            AMeDAS_STA_file = file
            AMeDAS_STA_df = read_csv_with_multiple_encodings("ame_master/" + AMeDAS_STA_file)
            break

    AMeDAS_STA_df["緯度"] = AMeDAS_STA_df.apply(lambda x: to_decimal(x["緯度(度)"], x["緯度(分)"]), axis=1)
    AMeDAS_STA_df["経度"] = AMeDAS_STA_df.apply(lambda x: to_decimal(x["経度(度)"], x["経度(分)"]), axis=1)
    AMeDAS_STA_df.to_csv("ame_master/" + AMeDAS_STA_file, index=False)
    return AMeDAS_STA_df

# ※ 原本取得局點資料的函式保持不變
def get_sta_from_JMA(pd="00"):
    cookies = {
        'AWSALB': 'osx6uR/c6KwcyMebiovRy3gAW+4aZLfcQPtU+6wJWwUnFm7qGQ3i1GXSVcIjBxrIJzLBkNrBn7CjRX6ixdUNbq1yVKy4/YrUzoF+GdpaoZYGXvHTkpFaB+WhoTB6',
        'AWSALBCORS': 'osx6uR/c6KwcyMebiovRy3gAW+4aZLfcQPtU+6wJWwUnFm7qGQ3i1GXSVcIjBxrIJzLBkNrBn7CjRX6ixdUNbq1yVKy4/YrUzoF+GdpaoZYGXvHTkpFaB+WhoTB6',
    }
    headers = {
        'Accept': 'text/html, */*; q=0.01',
        'Accept-Language': 'ja-JP,ja;q=0.9,zh-TW;q=0.8,zh;q=0.7,en-US;q=0.6,en;q=0.5',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Origin': 'https://www.data.jma.go.jp',
        'Referer': 'https://www.data.jma.go.jp/risk/obsdl/index.php',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }
    data = {
        'pd': pd,
    }
    response = requests.post('https://www.data.jma.go.jp/gmd/risk/obsdl/top/station', cookies=cookies, headers=headers, data=data)
    response.encoding = response.apparent_encoding
    return response.text

# -----------------------------------------------------
# 以下為修改後取得資料的主要函式
# 改用 pd.read_html 直接讀取網頁中的表格，並將 multi-index 欄位平坦化，
# 同時：
# 1. 當下載當月資料時，只下載到昨天（不下載未來日期）的資料
# 2. 若該月份的 CSV 檔在過去 24 小時內已有更新，則跳過下載
# -----------------------------------------------------
def download_weather_data(unique_sta_id):
    download_time_str = "ダウンロードした時刻：" + datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    header_columns = "年月日時,気温(℃),気温(℃),気温(℃),降水量(mm),降水量(mm),降水量(mm),日射量(MJ/㎡),日射量(MJ/㎡),日射量(MJ/㎡),天気,天気,天気,視程(km),視程(km),視程(km),雲量(10分比),雲量(10分比),雲量(10分比),現地気圧(hPa),現地気圧(hPa),現地気圧(hPa),海面気圧(hPa),海面気圧(hPa),海面気圧(hPa),相対湿度(％),相対湿度(％),相対湿度(％),風速(m/s),風速(m/s),風速(m/s),風速(m/s),風速(m/s),日照時間(時間),日照時間(時間),日照時間(時間),積雪(cm),積雪(cm),積雪(cm),降雪(cm),降雪(cm),降雪(cm)"
    header_extra1 = ",,,,,,,,,,,,,,,,,,,,,,,,,,,風向,風向,,,,,,,,,,"
    header_extra2 = ",,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質號碼,,品質情報,均質番号"

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    previous_month = current_month - 1 if current_month > 1 else 12
    previous_year = current_year if current_month > 1 else current_year - 1
    # 若今天尚未滿 12 號，則只下載上個月的資料；否則下載當月和（可能）上個月
    months_to_download = []
    if current_date.day < 12:
        months_to_download.append((previous_year, previous_month))
    else:
        months_to_download.append((current_year, current_month))
        # 如果當天早於 12 號，也可以考慮下載上個月（視需求決定）
        # months_to_download.append((previous_year, previous_month))
    
    # 定義 prec_no 的對應字典（依照兩個字的府/振興局名稱）
    prec_mapping = {
        "三重": "53",
        "上川": "12",
        "京都": "61",
        "佐賀": "85",
        "兵庫": "63",
        "十勝": "20",
        "千葉": "45",
        "和歌": "65",  # 和歌山取前兩字「和歌」
        "埼玉": "43",
        "大分": "83",
        "大阪": "62",
        "奈良": "64",
        "宗谷": "11",
        "宮城": "34",
        "宮崎": "87",
        "富山": "55",
        "山口": "81",
        "山形": "35",
        "山梨": "49",
        "岐阜": "52",
        "岡山": "66",
        "岩手": "33",
        "島根": "68",
        "広島": "67",
        "後志": "16",
        "徳島": "71",
        "愛媛": "73",
        "愛知": "51",
        "新潟": "54",
        "日高": "22",
        "東京": "44",
        "栃木": "41",
        "根室": "18",
        "檜山": "24",
        "沖縄": "91",
        "渡島": "23",
        "滋賀": "60",
        "熊本": "86",
        "留萌": "13",
        "石川": "56",
        "石狩": "14",
        "神奈": "46",  # 神奈川取前兩字
        "福井": "57",
        "福岡": "82",
        "福島": "36",
        "秋田": "32",
        "空知": "15",
        "群馬": "42",
        "ｵﾎ": "17"
        # 若站點中「都府県振興局」名稱不是完全一致，可考慮取前兩字再比對
    }
    
    for idx, row in unique_sta_id.iterrows():
        station_id = row["局ID"]
        station_name = row["局名"]
        # 這裡以局ID取部份作為 block_no，可依實際需求修改
        block_no = row.get("局ID", "00000")[1:]
        sta_type = str(row.get("局ID", "")).strip()[0]
        # 根據「都府県振興局」欄位取前兩字比對
        key = str(row.get("都府県振興局", "")).strip()[:2]
        prec_no = prec_mapping.get(key, "33")
        os.makedirs(f'{ROOT}/weather_data/{station_id}', exist_ok=True)
        
        for (year, month) in months_to_download:
            start_date = datetime(year, month, 1)
            # 若下載的是當月，僅下載到昨天為止
            if year == current_year and month == current_month:
                max_day = min((start_date.replace(day=28) + timedelta(days=4)).day, current_date.day - 1)
            else:
                next_month = start_date.replace(day=28) + timedelta(days=4)
                max_day = (next_month - timedelta(days=next_month.day)).day
            
            # 檢查該月份的 CSV 是否在 24 小時內已更新
            csv_path = f'{ROOT}/weather_data/{station_id}/{year}-{month}.csv.gz'
            if os.path.exists(csv_path):
                try:
                    with gzip.open(csv_path, 'rt', encoding='utf8') as f:
                        first_line = f.readline().strip()
                    if first_line.startswith("ダウンロードした時刻："):
                        prev_dt = datetime.strptime(first_line[len("ダウンロードした時刻："):], "%Y/%m/%d %H:%M:%S")
                        if (datetime.now() - prev_dt).total_seconds() < 24 * 3600:
                            print(f"Data for station {station_id} for {year}-{month} updated within last 24h. Skipping download.")
                            continue
                except Exception as e:
                    print(f"Error checking existing CSV for station {station_id} for {year}-{month}: {e}")
            
            output_rows = []
            
            for day in range(1, max_day + 1):
                url = f"https://www.data.jma.go.jp/stats/etrn/view/hourly_{sta_type}1.php?prec_no={prec_no}&block_no={block_no}&year={year}&month={month:02d}&day={day:02d}&view="
                print(f"Downloading data for station {station_id} on {year}-{month}-{day} with prec_no {prec_no}, url: {url}")
                df_day = pd.DataFrame()
                try:
                    df_day = pd.read_html(url)[0]
                    # 平坦化 multi-index 欄位
                    new_columns = []
                    for col in df_day.columns:
                        if isinstance(col, tuple):
                            colname = col[1] if (len(col) > 1 and col[1] and pd.notna(col[1])) else col[0]
                        else:
                            colname = col
                        colname = str(colname).replace("\n", "").strip()
                        new_columns.append(colname)
                    df_day.columns = new_columns
                    
                    rename_map = {
                        "時": "hour",
                        "現地": "pressure_local",
                        "海面": "pressure_sea",
                        "降水量 (mm)": "precipitation",
                        "気温 (℃)": "temperature",
                        "湿度 (％)": "humidity",
                        "風速": "wind_speed",
                        "風向": "wind_direction",
                        "日照 時間 (h)": "sunshine",
                        "全天 日射量 (MJ/㎡)": "solar",
                        "雪(cm)": "snow_depth",
                        "天気": "weather",
                        "雲量": "cloud",
                        "視程 (km)": "visibility"
                    }
                    df_day.rename(columns=rename_map, inplace=True)
                    
                    for i, r in df_day.iterrows():
                        try:
                            hour_val = int(r["hour"])
                        except Exception:
                            continue
                        timestamp = f"{year}/{month}/{day} {hour_val}:00:00"
                        temperature = r.get("temperature", "")
                        precipitation = r.get("precipitation", "")
                        solar = r.get("solar", "")
                        weather = r.get("weather", "")
                        visibility = r.get("visibility", "")
                        cloud = r.get("cloud", "")
                        pressure_local = r.get("pressure_local", "")
                        pressure_sea = r.get("pressure_sea", "")
                        humidity = r.get("humidity", "")
                        wind_speed = r.get("wind_speed", "")
                        wind_direction = r.get("wind_direction", "")
                        sunshine = r.get("sunshine", "")
                        snow_depth = r.get("snow_depth", "")
                        snowfall = ""  # 新來源沒有降雪欄位
                        
                        row_out = [
                            timestamp,
                            temperature, "8", "1",
                            precipitation, "8", "1",
                            solar, "0", "1",
                            weather, "0", "1",
                            visibility, "0", "1",
                            cloud, "0", "1",
                            pressure_local, "0", "1",
                            pressure_sea, "0", "1",
                            humidity, "0", "1",
                            wind_speed, "8", wind_direction, "8", "1",
                            sunshine, "0", "1",
                            snow_depth, "0", "1",
                            snowfall, "0", "1"
                        ]
                        if len(row_out) != 42:
                            print("Row length mismatch:", len(row_out))
                        output_rows.append(row_out)
                    time.sleep(1)
                except Exception as e:
                    print(f"Error processing {year}-{month}-{day} for station {station_id}: {e}")
                    continue
            
            if output_rows:
                csv_lines = []
                csv_lines.append(download_time_str)
                csv_lines.append("," + ",".join([station_name] * 42))
                csv_lines.append(header_columns)
                csv_lines.append(header_extra1)
                csv_lines.append(header_extra2)
                for row_vals in output_rows:
                    row_str = ",".join(str(v) for v in row_vals)
                    csv_lines.append(row_str)
                csv_content = "\n".join(csv_lines)
                with gzip.open(csv_path, 'wt', encoding="utf8") as f:
                    f.write(csv_content)
                print(f"Successfully wrote data for station {station_id} for {year}-{month}")
            else:
                print(f"No data found for station {station_id} for {year}-{month}")


In [ ]:
# 讀取合併後的站點清單，並對局ID 去除重複後下載資料
combined_df = pd.read_csv(f"{ROOT}stations/merged_sta_list.csv")
unique_sta_id = combined_df.drop_duplicates(subset="局ID")
download_weather_data(unique_sta_id)

Data for station s47401 for 2025-2 updated within last 24h. Skipping download.
Data for station a0002 for 2025-2 updated within last 24h. Skipping download.
Data for station a0003 for 2025-2 updated within last 24h. Skipping download.
Data for station s47402 for 2025-2 updated within last 24h. Skipping download.
Data for station a0005 for 2025-2 updated within last 24h. Skipping download.
Data for station a1051 for 2025-2 updated within last 24h. Skipping download.
Data for station a1054 for 2025-2 updated within last 24h. Skipping download.
Data for station a1203 for 2025-2 updated within last 24h. Skipping download.
Data for station a1284 for 2025-2 updated within last 24h. Skipping download.
Data for station a1285 for 2025-2 updated within last 24h. Skipping download.
Data for station a1512 for 2025-2 updated within last 24h. Skipping download.
Data for station a1528 for 2025-2 updated within last 24h. Skipping download.
Data for station a1546 for 2025-2 updated within last 24h. Ski